# US Tropical Cyclone data
---
Visualization of US storm data

In [1]:
import pandas as pd 
import numpy as np 
import seaborn as sns   
import plotly.express as px
import os

df = pd.read_excel('data/cleanusstorms.xlsx')




In [2]:
print(df.isnull().sum())


Year                            0
Seq                             0
Glide                         122
Disaster Group                  0
Disaster Subgroup               0
Disaster Type                   0
Disaster Subtype                0
Disaster Subsubtype           132
Event Name                     27
Country                         0
ISO                             0
Region                          0
Continent                       0
Location                        4
Origin                        132
Associated Dis                 80
Associated Dis2               122
OFDA Response                 132
Appeal                        103
Declaration                    76
Aid Contribution              132
Dis Mag Value                  58
Dis Mag Scale                   0
Latitude                      125
Longitude                     125
Local Time                    132
River Basin                   130
Start Year                      0
Start Month                    12
Start Day     

In [3]:
print(df.head())

   Year  Seq Glide Disaster Group Disaster Subgroup Disaster Type  \
0  1900    3   NaN        Natural    Meteorological         Storm   
1  1906    4   NaN        Natural    Meteorological         Storm   
2  1906    5   NaN        Natural    Meteorological         Storm   
3  1909    3   NaN        Natural    Meteorological         Storm   
4  1909    4   NaN        Natural    Meteorological         Storm   

   Disaster Subtype  Disaster Subsubtype Event Name  \
0  Tropical cyclone                  NaN        NaN   
1  Tropical cyclone                  NaN        NaN   
2  Tropical cyclone                  NaN        NaN   
3  Tropical cyclone                  NaN        NaN   
4  Tropical cyclone                  NaN        NaN   

                          Country  ... No Homeless Total Affected  \
0  United States of America (the)  ...         NaN            NaN   
1  United States of America (the)  ...         NaN            NaN   
2  United States of America (the)  ...         

In [4]:
# This df only looks at US Tropical cyclones identified in the last step.  Dropping columns that are not needed for this analysis.
#df.drop(['Disaster Group', 'Disaster Subgroup', 'Disaster Subtype', 'Country', 'Continent', 'ISO'], axis=1, inplace=True)

In [5]:
print(df.dtypes)

Year                            int64
Seq                             int64
Glide                          object
Disaster Group                 object
Disaster Subgroup              object
Disaster Type                  object
Disaster Subtype               object
Disaster Subsubtype           float64
Event Name                     object
Country                        object
ISO                            object
Region                         object
Continent                      object
Location                       object
Origin                        float64
Associated Dis                 object
Associated Dis2                object
OFDA Response                 float64
Appeal                         object
Declaration                    object
Aid Contribution              float64
Dis Mag Value                 float64
Dis Mag Scale                  object
Latitude                      float64
Longitude                     float64
Local Time                    float64
River Basin 

In [6]:
print(df.describe())

              Year         Seq  Disaster Subsubtype  Origin  OFDA Response  \
count   132.000000  132.000000                  0.0     0.0            0.0   
mean   1986.431818  273.689394                  NaN     NaN            NaN   
std      31.595534  203.224094                  NaN     NaN            NaN   
min    1900.000000    2.000000                  NaN     NaN            NaN   
25%    1971.500000   54.500000                  NaN     NaN            NaN   
50%    1995.500000  314.500000                  NaN     NaN            NaN   
75%    2009.000000  432.000000                  NaN     NaN            NaN   
max    2021.000000  782.000000                  NaN     NaN            NaN   

       Aid Contribution  Dis Mag Value    Latitude   Longitude  Local Time  \
count               0.0      74.000000    7.000000    7.000000         0.0   
mean                NaN    5649.527027   76.195071 -187.460000         NaN   
std                 NaN   31682.088392  114.208525  272.423446 

In [7]:
storms_per_year = df.groupby('Year')['Event Name'].count()

fig = px.line(storms_per_year, labels={'value': 'Number of Cyclones', 'Year': 'Year'}, title='US Tropical Cyclones Frequency Over Time')
fig.update_xaxes(range=[1947, 2025])
fig.update_traces(mode='lines+markers')
fig.show()



In [8]:
print(df['Region'].value_counts())

Region
Gulf Coast    90
East Coast    26
Other         13
Island         2
West Coast     1
Name: count, dtype: int64


In [9]:
# Frequency of Events by Region
events_by_region = df.groupby('Region')['Event Name'].nunique().reset_index(name='Number of Events')

# Visualization
fig = px.bar(events_by_region, x='Region', y='Number of Events', title='Frequency of Tropical Cyclones by Region')
fig.show()


In [10]:
base_year_cpi = df['CPI'].max()

# Adjust damages to the base year's CPI value
df['Adjusted Damages'] = df.apply(lambda row: (row['Total Damages (\'000 US$)'] * base_year_cpi) / row['CPI'] if pd.notnull(row['Total Damages (\'000 US$)']) else None, axis=1)

# Total Adjusted Damages by Region
annual_damages_by_region = df.groupby(['Start Year', 'Region'])['Adjusted Damages'].sum().reset_index()


In [11]:
fig = px.line(annual_damages_by_region, x='Start Year', y='Adjusted Damages', color='Region',
              title='Annual Damages by Region (Adjusted for Inflation)',
              labels={'Adjusted Damages': 'Adjusted Damages (\'000 US$)', 'Start Year': 'Year'})
fig.update_xaxes(range=[1947, 2025])
fig.show()
